In [ ]:
## test CnrR1

%reload_ext autoreload
%autoreload 2
import os
import sys
from cnr_r1 import CnrR1
from hiseq.utils.file import list_file


data_dir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/data/fruitfly/'

fq1 = list_file(data_dir, '*Piwi*K4*1.fq.gz')
fq2 = list_file(data_dir, '*Piwi*K4*2.fq.gz')
outdir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/output/fruitfly'

args = {
    'fq1': fq1[1],
    'fq2': fq2[1],
    'genome': 'dm6',
    'outdir': outdir,
    'threads': 12,
    'parallel_jobs': 2
}

CnrR1(**args).run()




In [1]:
## test CnrRn

%reload_ext autoreload
%autoreload 2
import os
import sys
from cnr_r1 import CnrR1
from cnr_rn import CnrRn
from hiseq.utils.file import list_file
from hiseq.utils.utils import read_hiseq, Config, list_hiseq_file
from utils import qc_frip

# data_dir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/data/fruitfly/'


# fq1 = list_file(data_dir, '*Piwi*IgG*1.fq.gz')
# fq2 = list_file(data_dir, '*Piwi*IgG*2.fq.gz')

# outdir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/output/fruitfly'

# args = {
#     'fq1': fq1,
#     'fq2': fq2,
#     'genome': 'dm6',
#     'outdir': outdir,
#     'threads': 12,
#     'parallel_jobs': 4
# }

# CnrRn(**args).run()


c = '/data/yulab/wangming/work/yu_2021/projects/20210421_lxh_CnR/results/CnR_v2/CnR_Nos_DaGal4XshPiwi_6-8h_IgG/config/config.yaml'
args = Config().load(c)
a = CnrRn(**args)
# a.run_pipe_r1()
a.run()

# rep_dir = '/data/yulab/wangming/work/yu_2021/projects/20210421_lxh_CnR/results/CnR_v2/CnR_Nos_DaGal4XshPiwi_6-8h_IgG_rep1'
# k = 'bam'
# list_hiseq_file(rep_dir, k, 'r1')



# x = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/output/fruitfly/CUT_and_RUN_Piwi_RNAi_embryo_3_4h_H3K4me3/'
# pd = read_hiseq(x)
# qc_frip(x)

[2021-06-15 23:17:41 INFO] symlink_file() skipped, dest exists: /data/yulab/wangming/work/yu_2021/projects/20210421_lxh_CnR/results/CnR_v2/CnR_Nos_DaGal4XshPiwi_6-8h_IgG_rep1/raw_data/CnR_Nos_DaGal4XshPiwi_6-8h_IgG_rep1_1.fq.gz
[2021-06-15 23:17:41 INFO] symlink_file() skipped, dest exists: /data/yulab/wangming/work/yu_2021/projects/20210421_lxh_CnR/results/CnR_v2/CnR_Nos_DaGal4XshPiwi_6-8h_IgG_rep1/raw_data/CnR_Nos_DaGal4XshPiwi_6-8h_IgG_rep1_2.fq.gz
[2021-06-15 23:17:41 INFO] trim() skipped, file exists: /data/yulab/wangming/work/yu_2021/projects/20210421_lxh_CnR/results/CnR_v2/CnR_Nos_DaGal4XshPiwi_6-8h_IgG_rep1/clean_data/CnR_Nos_DaGal4XshPiwi_6-8h_IgG_rep1/CnR_Nos_DaGal4XshPiwi_6-8h_IgG_rep1_1.fq.gz
[2021-06-15 23:17:41 INFO] symlink_file() skipped, dest exists: /data/yulab/wangming/work/yu_2021/projects/20210421_lxh_CnR/results/CnR_v2/CnR_Nos_DaGal4XshPiwi_6-8h_IgG_rep1/clean_data/CnR_Nos_DaGal4XshPiwi_6-8h_IgG_rep1_1.fq.gz
[2021-06-15 23:17:41 INFO] symlink_file() skipped, dest 

In [ ]:
# test CnrRx # design

%reload_ext autoreload
%autoreload 2
import os
import sys
from cnr_r1 import CnrR1
from cnr_rn import CnrRn
from cnr_rx import CnrRx
from hiseq.utils.file import list_file
from hiseq.utils.utils import read_hiseq, print_dict
from utils import qc_frip
data_dir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/data/fruitfly/'

ip_fq1 = list_file(data_dir, '*White*K4*1.fq.gz')
ip_fq2 = list_file(data_dir, '*White*K4*2.fq.gz')
input_fq1 = list_file(data_dir, '*White*IgG*1.fq.gz')
input_fq2 = list_file(data_dir, '*White*IgG*2.fq.gz')

outdir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/output/fruitfly'

args = {
    'ip_fq1': ip_fq1,
    'ip_fq2': ip_fq2,
    'input_fq1': input_fq1,
    'input_fq2': input_fq2,
    'genome': 'dm6',
    'outdir': outdir,
    'threads': 12,
    'parallel_jobs': 4,
    'gene_bed': '/home/wangming/data/genome/dm6/annotation_and_repeats/dm6.ensembl.bed',
}

CnrRx(**args).run()


In [88]:
import pandas as pd


def parse_report(in_report, out_stat, topN=10, tax_level='G'):
        """
        Parameters
        ----------
        in_report: str
            The --report output of kraken2
            
        out_stat: str
            The file saving the simplified report, by specific tax level
            
        topN: int
            The top N taxon, default: 100
            
        tax_level: str
            The specific tax level to show, default: ['G'],
            options: [D, K, P, C, O, F, G, S]
        
        parse Kraken2 report (--report)
        Inspect the output of kraken2 report file
        pandas table
        
        col-1: Percentage of fragments covered by the clade rooted at this taxon
        col-2: Number of fragments covered by the clade rooted at this taxon
        col-3: Number of fragments assigned directly to this taxon
        col-4: A rank code, indicating (U)nclassified, (R)oot, (D)omain, 
               (K)ingdom, (P)hylum, (C)lass, (O)rder, (F)amily, (G)enus,
               or (S)pecies. Taxa that are not at any of these 10 ranks have 
               a rank code that is formed by using the rank code of the closest
               ancestor rank with a number indicating the distance from that rank.
               E.g., "G2" is a rank code indicating a taxon is between genus and 
               species and the grandparent taxon is at the genus rank.
        col-5: NCBI taxonomic ID number
        col-6: Indented scientific name
        
        # choose the top species, by col-3, read directly in taxon
        """
        s = ['pct', 'reads_in_clade', 'reads_in_tax', 'code', 'taxid', 'name']
        try:
            df1 = pd.read_csv(in_report, '\t', names=s)
        except IOError as e:
            log.error(e)
            return None
        # choose Geneus (G); root, unclassified
        df_un = df1.loc[df1['code'] == 'U'] # unclassified
        df_root = df1.loc[df1['code'] == 'R'] # root
        df_tax = df1.loc[df1['code'].isin([tax_level]), ] # eg: G
        # choose top ranked taxon
        df_tax = df_tax.sort_values(['reads_in_tax'], ascending=False)
        # sub-sample
        df2 = df_tax.loc[0:topN,]
#         # combine
#         df = pd.concate([df_un, df2])
        ## remove white spaces
#         df['name'] = df['name'].str.strip()
#         # add sample name
#         df['sample'] = file_prefix(x) # prefix
#         # get total
#         n_hit = df_root.loc[:, 'reads_in_clade']
#         n_unhit = df_un.loc[:, 'reads_in_clade']
#         # adt pct
#         df['reads_hit'] = int(n_hit) # root
#         df['reads_total'] = int(n_hit) + int(n_unhit) # root + unclassified
#         df['hit_pct'] = df['reads_in_tax'] / int(n_hit) * 100
#          # sub-sample
#         df3 = df.loc[:,['sample', 'name', 'reads_in_tax', 'hit_pct', 'reads_hit', 'reads_total']]        
#         # custome options
#         with pd.option_context('display.expand_frame_repr', False, 'display.max_colwidth', 20):
#             print(df3)
#         # save to file
#         df3.to_csv(out_stat, sep='\t', index=False)
#         return df3

in_report = '/data/yulab/wangming/work/devel_pipeline/hiseq/qc/aaaa.kraken2.report'
out_stat = '/data/yulab/wangming/work/devel_pipeline/hiseq/qc/aaaa.kraken2.stat'
topN = 10
# parse_report(in_report, out_stat)
tax_level = 'G'
s = ['pct', 'reads_in_clade', 'reads_in_tax', 'code', 'taxid', 'name']
try:
    df1 = pd.read_csv(in_report, '\t', names=s)
except IOError as e:
    log.error(e)
# choose Geneus (G); root, unclassified
df_un = df1.loc[df1['code'] == 'U'] # unclassified
df_root = df1.loc[df1['code'] == 'R'] # root
df_tax = df1.loc[df1['code'].isin([tax_level]), ] # eg: G
# choose top ranked taxon
df_tax = df_tax.sort_values(['reads_in_tax'], ascending=False)
# sub-sample
df2 = df_tax.iloc[0:topN,]
df2

,pct,reads_in_clade,reads_in_tax,code,taxid,name
1988,0.02,825,617,G,2132,Spiroplasma
2418,1.02,40737,354,G,570,Klebsiella
2928,0.05,1834,311,G,286,Pseudomonas
948,0.08,3298,265,G,1883,Streptomyces
1310,0.02,708,236,G,1763,Mycobacterium
5925,0.01,235,235,G,160798,Marinitoga
6563,0.00,192,188,G,181083,Chlorovirus
187,0.35,13945,138,G,1386,Bacillus
4335,0.00,132,130,G,34008,Rhodovulum
4181,0.01,308,123,G,773,Bartonella


In [91]:
 all([False])

False

In [ ]:
# test CnrRd # design
%reload_ext autoreload
%autoreload 2
from cnr_rd import CnrRd

data_dir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/data/fruitfly/'

ip_fq1 = list_file(data_dir, '*Piwi*K4*1.fq.gz')
ip_fq2 = list_file(data_dir, '*Piwi*K4*2.fq.gz')

input_fq1 = list_file(data_dir, '*Piwi*IgG*1.fq.gz')
input_fq2 = list_file(data_dir, '*Piwi*IgG*2.fq.gz')
outdir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/output/fruitfly'

args = {
    'design': 'aaa.toml',
    'fq_dir': data_dir,
    'ip_fq1': ip_fq1,
    'ip_fq2': ip_fq2,
    'input_fq1': input_fq1,
    'input_fq2': input_fq2
}


args = {
    'design': 'aaa.toml',
    'fq_dir': data_dir,
    'ip': ['Piwi_RNAi_embryo_3_4h_H3K4me3', 'White_RNAi_embryo_3_4h_H3K4me3'],
    'input': ['Piwi_RNAi_embryo_3_4h_IgG'],
}

a = CnrRd(**args)

#a.check_fx_args()
df = a.run()

In [ ]:

from difflib import SequenceMatcher

string1 = "apple pie available"
string2 = "come have some apple pies"

# match = SequenceMatcher(None, string1, string2).find_longest_match(0, len(string1), 0, len(string2))

# print(match)  # -> Match(a=0, b=15, size=9)
# print(string1[match.a: match.a + match.size])  # -> apple pie
# print(string2[match.b: match.b + match.size])  # -> apple pie

# string1[match.a:match.a+match.size]

# dir(match)


## Temp
def find_longest_common_str(s1, s2):
    if isinstance(s1, str) and isinstance(s2, str):
        m = SequenceMatcher(None, s1, s2) # match
        l = m.find_longest_match(0, len(s1), 0, len(s2))
        out = s1[l.a: l.a + l.size]
    else:
        log.error('only support str, got s1={} s2={}'.type(
            type(s1).__name__, type(s2).__name__))
        out = None
    return out



find_longest_common_str('abc', 'efgab')
